In [1]:
import os
from collections import deque
from dotenv import load_dotenv
from google import genai
import pandas as pd
from pydantic import BaseModel, Field
import time
from typing import Annotated, Literal

folder_path = 'data/global/'

ieee_file = folder_path + 'ieee_global.csv'
scopus_file = folder_path + 'scopus_global.csv'
wos_file = folder_path + 'wos_global.xls'

# IEEE Xplore

In [2]:
df_ieee = pd.read_csv(ieee_file)
df_ieee

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,The Impact of National Land Cover and Soils Da...,A. Pacheco; H. McNairn; A. Mahmoodi; C. Champa...,"Agriculture and Agri-Food Canada, Ottawa, ON, ...",IEEE Journal of Selected Topics in Applied Ear...,20 May 2017,2015,8.0,11.0,5281,5293,...,NaN,14.0,NaN,40.0,Crown,3 Jun 2015,NaN,NaN,IEEE,IEEE Journals
1,Evaluation of Land Surface Model Against Smap ...,K. Das; J. Singh; J. Hazra; S. Kalyanaraman,"IBM Research, India; IBM Research, India; IBM ...",IGARSS 2018 - 2018 IEEE International Geoscien...,4 Nov 2018,2018,NaN,NaN,92,95,...,NaN,5.0,NaN,12.0,IEEE,4 Nov 2018,NaN,NaN,IEEE,IEEE Conferences
2,A Comprehensive Survey of Classification Algor...,C. Chandana; G. Parthasarathy,"Research Schalor, Computer Science and Enginee...",2020 IEEE International Conference on Technolo...,6 Oct 2021,2020,NaN,NaN,1,5,...,NaN,8.0,NaN,26.0,IEEE,6 Oct 2021,NaN,NaN,IEEE,IEEE Conferences
3,Land degradation estimation from global and na...,N. Kussul; A. Kolotii; A. Shelestov; B. Yailym...,"Space Research Institute NASU-SSAU, Kyiv, Ukra...",2017 9th IEEE International Conference on Inte...,7 Nov 2017,2017,1.0,NaN,383,386,...,NaN,11.0,NaN,21.0,IEEE,7 Nov 2017,NaN,NaN,IEEE,IEEE Conferences
4,A global dataset of noiseless time-series vege...,A. Kotera; S. Berberoglu; T. Nagano; M. A. Cullu,"Research Department, Research Institute for Hu...",2015 Fourth International Conference on Agro-G...,10 Sep 2015,2015,NaN,NaN,173,177,...,NaN,NaN,NaN,6.0,IEEE,10 Sep 2015,NaN,NaN,IEEE,IEEE Conferences
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,FFM: Flood Forecasting Model Using Federated L...,M. S. Farooq; R. Tehseen; J. N. Qureshi; U. Om...,"Department of Computer Science, University of ...",IEEE Access,16 Mar 2023,2023,11.0,NaN,24472,24483,...,NaN,31.0,NaN,54.0,CCBYNCND,6 Mar 2023,NaN,NaN,IEEE,IEEE Journals
114,DynaES: Dynamic Energy Scheduling for Energy H...,J. Hao; E. Oni; I. K. Kim; L. Ramaswamy,"School of Computing, The University of Georgia...","2023 IEEE International Performance, Computing...",18 Oct 2023,2023,NaN,NaN,365,374,...,NaN,1.0,NaN,34.0,IEEE,18 Oct 2023,NaN,NaN,IEEE,IEEE Conferences
115,Segmentation and detection from organised 3D p...,J. L. Louedec; H. A. Montes; T. Duckett; G. Ci...,University of Lincoln; University of Lincoln; ...,2020 IEEE/CVF Conference on Computer Vision an...,28 Jul 2020,2020,NaN,NaN,285,293,...,NaN,13.0,NaN,20.0,IEEE,28 Jul 2020,NaN,NaN,IEEE,IEEE Conferences
116,Fruit Picker Activity Recognition with Wearabl...,J. J. Dabrowski; A. Rahman,"Data61, CSIRO, Brisbane, Australia; Data61, CS...",2023 International Joint Conference on Neural ...,2 Aug 2023,2023,NaN,NaN,1,8,...,NaN,1.0,NaN,34.0,IEEE,2 Aug 2023,NaN,NaN,IEEE,IEEE Conferences


In [3]:
df_ieee = df_ieee[['DOI', 'Document Title', 'Authors', 'Publication Year', 'Abstract', 'Article Citation Count']]

df_ieee = df_ieee.rename(columns={
    'Document Title': 'Title',
    'Publication Year': 'Year',
    'Article Citation Count': 'Citations'
})

df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/JSTARS.2015.2417832,The Impact of National Land Cover and Soils Da...,A. Pacheco; H. McNairn; A. Mahmoodi; C. Champa...,2015,"To ensure sustainable agriculture production, ...",14.0
1,10.1109/IGARSS.2018.8518014,Evaluation of Land Surface Model Against Smap ...,K. Das; J. Singh; J. Hazra; S. Kalyanaraman,2018,Soil moisture and temperature are key inputs t...,5.0
2,10.1109/TEMSMET51618.2020.9557403,A Comprehensive Survey of Classification Algor...,C. Chandana; G. Parthasarathy,2020,As we are aware that agriculture is one of the...,8.0
3,10.1109/IDAACS.2017.8095109,Land degradation estimation from global and na...,N. Kussul; A. Kolotii; A. Shelestov; B. Yailym...,2017,"In this paper, we investigate global and natio...",11.0
4,10.1109/Agro-Geoinformatics.2015.7248132,A global dataset of noiseless time-series vege...,A. Kotera; S. Berberoglu; T. Nagano; M. A. Cullu,2015,Time-series vegetation and water indices deriv...,NaN
...,...,...,...,...,...,...
113,10.1109/ACCESS.2023.3252896,FFM: Flood Forecasting Model Using Federated L...,M. S. Farooq; R. Tehseen; J. N. Qureshi; U. Om...,2023,Floods are one of the most common natural disa...,31.0
114,10.1109/IPCCC59175.2023.10253869,DynaES: Dynamic Energy Scheduling for Energy H...,J. Hao; E. Oni; I. K. Kim; L. Ramaswamy,2023,Low-cost sensors and IoT technologies have fac...,1.0
115,10.1109/CVPRW50498.2020.00040,Segmentation and detection from organised 3D p...,J. L. Louedec; H. A. Montes; T. Duckett; G. Ci...,2020,Autonomous harvesting is becoming an important...,13.0
116,10.1109/IJCNN54540.2023.10191571,Fruit Picker Activity Recognition with Wearabl...,J. J. Dabrowski; A. Rahman,2023,In this paper we present a novel application o...,1.0


In [4]:
df_ieee = df_ieee.dropna(subset=['DOI'])
df_ieee = df_ieee.dropna(subset=['Citations'])
df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/JSTARS.2015.2417832,The Impact of National Land Cover and Soils Da...,A. Pacheco; H. McNairn; A. Mahmoodi; C. Champa...,2015,"To ensure sustainable agriculture production, ...",14.0
1,10.1109/IGARSS.2018.8518014,Evaluation of Land Surface Model Against Smap ...,K. Das; J. Singh; J. Hazra; S. Kalyanaraman,2018,Soil moisture and temperature are key inputs t...,5.0
2,10.1109/TEMSMET51618.2020.9557403,A Comprehensive Survey of Classification Algor...,C. Chandana; G. Parthasarathy,2020,As we are aware that agriculture is one of the...,8.0
3,10.1109/IDAACS.2017.8095109,Land degradation estimation from global and na...,N. Kussul; A. Kolotii; A. Shelestov; B. Yailym...,2017,"In this paper, we investigate global and natio...",11.0
5,10.1109/MetroAgriFor52389.2021.9628717,Improving accuracy and efficiency in plant det...,L. Ohnemüller; A. Briassouli,2021,Detecting plants in images is central in preci...,2.0
...,...,...,...,...,...,...
111,10.1109/I2CT61223.2024.10544089,Paddy Leaf Disease Detection Using Ensemble St...,S. B. V. M; D. Badawadagi; A. P. Bidargaddi; S...,2024,"Paddy leaves, fundamental components of rice p...",1.0
113,10.1109/ACCESS.2023.3252896,FFM: Flood Forecasting Model Using Federated L...,M. S. Farooq; R. Tehseen; J. N. Qureshi; U. Om...,2023,Floods are one of the most common natural disa...,31.0
114,10.1109/IPCCC59175.2023.10253869,DynaES: Dynamic Energy Scheduling for Energy H...,J. Hao; E. Oni; I. K. Kim; L. Ramaswamy,2023,Low-cost sensors and IoT technologies have fac...,1.0
115,10.1109/CVPRW50498.2020.00040,Segmentation and detection from organised 3D p...,J. L. Louedec; H. A. Montes; T. Duckett; G. Ci...,2020,Autonomous harvesting is becoming an important...,13.0


# Scopus

In [5]:
df_scopus = pd.read_csv(scopus_file)
df_scopus

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"L., Qian, Long; X., Yu, Xingjiao; Z., Zhang, Z...","Qian, Long (57226014243); Yu, Xingjiao (581133...",57226014243; 58113326000; 55721751200; 5571434...,Assessing and improving the high uncertainty o...,2024,Science of the Total Environment,957,NaN,177344,NaN,...,00489697; 18791026,NaN,STEVA,39521074.0,English,Article,Final,NaN,Scopus,2-s2.0-85208809052
1,"J., Strobl, Josef; A., Nazarkulova, Ainura","Strobl, Josef (7003410553); Nazarkulova, Ainur...",7003410553; 49964155400,Informing Distributed Hydrological Modeling wi...,2024,International Journal of Geoinformatics,20,12,NaN,1,...,16866576; 26730014,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85215435657
2,"L., Ousayd, Lahcen; T., Epule, Terence; S., Be...","Ousayd, Lahcen (58537090500); Epule, Terence (...",58537090500; 56019255900; 55027313800; 5638157...,"The impact of precipitation, temperature, and ...",2024,Agriculture and Food Security,13,1,55,NaN,...,20487010,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85214098864
3,"S., Fan, Shunxiang; T., Newbold, Tim; T., Tsch...","Fan, Shunxiang (57202250911); Newbold, Tim (22...",57202250911; 22135322100; 7003472839; 58929876...,Impact of Crop Type on Biodiversity Globally,2024,Global Change Biology,30,12,e70005,NaN,...,13541013; 13652486,NaN,NaN,39714079.0,English,Article,Final,NaN,Scopus,2-s2.0-85212814425
4,"A., Panwar, Anita; S.J., Nanda, Satyasai Jagan...","Panwar, Anita (55201377600); Nanda, Satyasai J...",55201377600; 26325893500,Distributed Clustering in Wireless Sensor Netw...,2024,SN Computer Science,5,8,1114,NaN,...,26618907; 2662995X,NaN,NaN,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85211176505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,"M., Wahabzada, Mirwaes; A.K., Mahlein, Anne Ka...","Wahabzada, Mirwaes (36563131100); Mahlein, Ann...",36563131100; 25630875100; 56234566100; 7006069...,Metro maps of plant disease dynamics-automated...,2015,PLOS ONE,10,1,e0116902,NaN,...,19326203,NaN,POLNC,25621489.0,English,Article,Final,All Open Access; Gold Open Access; Green Accep...,Scopus,2-s2.0-84921900726
368,"J., Nawai, J.; M.A., Gusyev, Maksym A.; A., Ha...","Nawai, J. (57215423607); Gusyev, Maksym A. (36...",57215423607; 36150115400; 57201565982; 7403989544,Flood and drought assessment with dam infrastr...,2015,NaN,NaN,NaN,NaN,1607,...,NaN,9780987214355,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85080965844
369,"K., Navarathinam, K.; M.A., Gusyev, Maksym A.;...","Navarathinam, K. (57215409976); Gusyev, Maksym...",57215409976; 36150115400; 57201565982; 7801482...,Agricultural flood and drought risk reduction ...,2015,NaN,NaN,NaN,NaN,1600,...,NaN,9780987214355,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85080916080
370,"F., Li, Fuqin; D., Jupp, D.; M., Thankappan, M...","Li, Fuqin (8308279800); Jupp, D. (7003479169);...",8308279800; 7003479169; 23013364100; 571927024...,Land surface brightness temperature retrieved ...,2015,NaN,NaN,NaN,NaN,2332,...,NaN,9780987214355,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85080884119


In [6]:
df_scopus = df_scopus[['DOI', 'Title', 'Authors', 'Year', 'Abstract', 'Cited by']]

df_scopus = df_scopus.rename(columns={
    'Cited by': 'Citations'
})

df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.scitotenv.2024.177344,Assessing and improving the high uncertainty o...,"L., Qian, Long; X., Yu, Xingjiao; Z., Zhang, Z...",2024,Gross Primary Productivity (GPP) is a crucial ...,4
1,10.52939/ijg.v20i12.3763,Informing Distributed Hydrological Modeling wi...,"J., Strobl, Josef; A., Nazarkulova, Ainura",2024,Continuity of water supply is critical for sus...,0
2,10.1186/s40066-024-00509-w,"The impact of precipitation, temperature, and ...","L., Ousayd, Lahcen; T., Epule, Terence; S., Be...",2024,Background: Climate change has devastating imp...,5
3,10.1111/gcb.70005,Impact of Crop Type on Biodiversity Globally,"S., Fan, Shunxiang; T., Newbold, Tim; T., Tsch...",2024,The negative impact of agricultural land on bi...,6
4,10.1007/s42979-024-03446-4,Distributed Clustering in Wireless Sensor Netw...,"A., Panwar, Anita; S.J., Nanda, Satyasai Jagan...",2024,The main limitation of the Fuzzy C-Means techn...,3
...,...,...,...,...,...,...
367,10.1371/journal.pone.0116902,Metro maps of plant disease dynamics-automated...,"M., Wahabzada, Mirwaes; A.K., Mahlein, Anne Ka...",2015,Understanding the response dynamics of plants ...,110
368,NaN,Flood and drought assessment with dam infrastr...,"J., Nawai, J.; M.A., Gusyev, Maksym A.; A., Ha...",2015,"In Fiji, frequent flood and drought events req...",9
369,NaN,Agricultural flood and drought risk reduction ...,"K., Navarathinam, K.; M.A., Gusyev, Maksym A.;...",2015,"In this study, we investigate the role of a pr...",14
370,NaN,Land surface brightness temperature retrieved ...,"F., Li, Fuqin; D., Jupp, D.; M., Thankappan, M...",2015,Land Surface Temperature (T<inf>s</inf>) is an...,1


In [7]:
df_scopus = df_scopus.dropna(subset=['DOI'])
df_scopus = df_scopus.loc[df_scopus['Citations'] > 0]
df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1016/j.scitotenv.2024.177344,Assessing and improving the high uncertainty o...,"L., Qian, Long; X., Yu, Xingjiao; Z., Zhang, Z...",2024,Gross Primary Productivity (GPP) is a crucial ...,4
2,10.1186/s40066-024-00509-w,"The impact of precipitation, temperature, and ...","L., Ousayd, Lahcen; T., Epule, Terence; S., Be...",2024,Background: Climate change has devastating imp...,5
3,10.1111/gcb.70005,Impact of Crop Type on Biodiversity Globally,"S., Fan, Shunxiang; T., Newbold, Tim; T., Tsch...",2024,The negative impact of agricultural land on bi...,6
4,10.1007/s42979-024-03446-4,Distributed Clustering in Wireless Sensor Netw...,"A., Panwar, Anita; S.J., Nanda, Satyasai Jagan...",2024,The main limitation of the Fuzzy C-Means techn...,3
5,10.1016/j.aiia.2024.11.005,Enhancing crop yield prediction in Senegal usi...,"M.A., Razavi, Mohammad Amin; A.P., Nejadhashem...",2024,"In this study, we employ advanced data-driven ...",7
...,...,...,...,...,...,...
364,10.1186/s12992-015-0087-y,Maternal mortality: A cross-sectional study in...,"S., Sajedinejad, Sima; R., Majdzadeh, Reza; A....",2015,Background: Although most of maternal deaths a...,50
365,10.5194/hess-19-823-2015,A high-resolution global-scale groundwater model,"I.E., de Graaf, Inge E.M.; E.H., Sutanudjaja, ...",2015,Groundwater is the world's largest accessible ...,170
366,10.1111/geb.12243,An analysis of methodological and spatial diff...,"W.B., Anderson, Weston B.; L., You, Liangzhi; ...",2015,Aim: Agricultural practices have dramatically ...,47
367,10.1371/journal.pone.0116902,Metro maps of plant disease dynamics-automated...,"M., Wahabzada, Mirwaes; A.K., Mahlein, Anne Ka...",2015,Understanding the response dynamics of plants ...,110


# Web of Science

In [8]:
df_wos = pd.read_excel(wos_file)
df_wos

,Publication Type,Authors,Book Authors,Group Authors,Book Group Authors,Researcher Ids,ORCIDs,Book Editors,Author - Arabic,Grant Principal Investigator,...,Copyright,Degree Name,Institution Address,Institution,Dissertation and Thesis Subjects,Author Keywords,Indexed Date,UT (Unique ID),Pubmed Id,Unnamed: 78
0,J,"Tang, Fiona H. M.; Lenzen, Manfred; McBratney,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-29,WOS:000634737700001,NaN,NaN
1,J,"Allen, Toph; Murray, Kris A.; Zambrana-Torreli...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2017-10-24,WOS:000413573000019,29066781.0,NaN
2,J,"Dainese, Matteo; Martin, Emily A.; Aizen, Marc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-01,WOS:000491132700077,31663019.0,NaN
3,J,"Janssens-Maenhout, G.; Crippa, M.; Guizzardi, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,WOS:000362971000030,NaN,NaN
4,J,"Dai, Zhongmin; Su, Weiqin; Chen, Huaihai; Barb...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-01,WOS:000437284700016,29645398.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,J,"Hughes, Daryl; Birkinshaw, Steve; Parkin, Geof...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-01,WOS:001101387400001,NaN,NaN
507,J,"Meng, Yao; Yoon, Sook; Han, Shujie; Fuentes, A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-26,ZOOREC:ZOOR16002010831,NaN,NaN
508,J,"Darwell, Clive T.; Wanchana, Samart; Ruanjaich...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-26,WOS:000795718600001,35574109.0,NaN
509,C,"Akkak, A.",NaN,NaN,NaN,NaN,NaN,"Akbulut, M; Mitra, S; Reinhardt, DH; Ikten, H;...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-01,WOS:001207667600062,NaN,NaN


In [9]:
df_wos = df_wos[['DOI', 'Article Title', 'Authors', 'Publication Year', 'Abstract', 'Times Cited, All Databases']]

df_wos = df_wos.rename(columns={
    'Article Title': 'Title',
    'Publication Year': 'Year',
    'Times Cited, All Databases': 'Citations'
})

df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1038/s41561-021-00712-5,Risk of pesticide pollution at the global scale,"Tang, Fiona H. M.; Lenzen, Manfred; McBratney,...",2021,Pesticides are widely used to protect food pro...,852
1,10.1038/s41467-017-00923-8,Global hotspots and correlates of emerging zoo...,"Allen, Toph; Murray, Kris A.; Zambrana-Torreli...",2017,Zoonoses originating from wildlife represent a...,780
2,10.1126/sciadv.aax0121,A global synthesis reveals biodiversity-mediat...,"Dainese, Matteo; Martin, Emily A.; Aizen, Marc...",2019,Human land use threatens global biodiversity a...,729
3,10.5194/acp-15-11411-2015,HTAP_v2.2: a mosaic of regional and global emi...,"Janssens-Maenhout, G.; Crippa, M.; Guizzardi, ...",2015,The mandate of the Task Force Hemispheric Tran...,688
4,10.1111/gcb.14163,Long-term nitrogen fertilization decreases bac...,"Dai, Zhongmin; Su, Weiqin; Chen, Huaihai; Barb...",2018,Long-term elevated nitrogen (N) input from ant...,654
...,...,...,...,...,...,...
506,10.1080/15715124.2023.2278678,An innovative hydrological model for the spars...,"Hughes, Daryl; Birkinshaw, Steve; Parkin, Geof...",2025,Tropical river basins - crucial components of ...,0
507,NaN,Improving Known-Unknown Cattle's Face Recognit...,"Meng, Yao; Yoon, Sook; Han, Shujie; Fuentes, A...",2023,"Simple Summary Over the years, the identificat...",0
508,10.3389/fpls.2022.781153,riceExplorer: Uncovering the Hidden Potential ...,"Darwell, Clive T.; Wanchana, Samart; Ruanjaich...",2022,"Agricultural crop breeding programs, particula...",0
509,10.17660/ActaHortic.2020.1299.63,Genetic characterization of date palm (Phoenix...,"Akkak, A.",2020,Algeria is one of the leading date producers i...,0


In [10]:
df_wos = df_wos.dropna(subset=['DOI', 'Abstract'])
df_wos = df_wos.loc[df_wos['Year'] != 2025]
df_wos = df_wos.loc[df_wos['Citations'] > 0]
df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1038/s41561-021-00712-5,Risk of pesticide pollution at the global scale,"Tang, Fiona H. M.; Lenzen, Manfred; McBratney,...",2021,Pesticides are widely used to protect food pro...,852
1,10.1038/s41467-017-00923-8,Global hotspots and correlates of emerging zoo...,"Allen, Toph; Murray, Kris A.; Zambrana-Torreli...",2017,Zoonoses originating from wildlife represent a...,780
2,10.1126/sciadv.aax0121,A global synthesis reveals biodiversity-mediat...,"Dainese, Matteo; Martin, Emily A.; Aizen, Marc...",2019,Human land use threatens global biodiversity a...,729
3,10.5194/acp-15-11411-2015,HTAP_v2.2: a mosaic of regional and global emi...,"Janssens-Maenhout, G.; Crippa, M.; Guizzardi, ...",2015,The mandate of the Task Force Hemispheric Tran...,688
4,10.1111/gcb.14163,Long-term nitrogen fertilization decreases bac...,"Dai, Zhongmin; Su, Weiqin; Chen, Huaihai; Barb...",2018,Long-term elevated nitrogen (N) input from ant...,654
...,...,...,...,...,...,...
480,10.1016/j.canrad.2023.06.016,Real world data in radiotherapy: A data farmin...,"Guihard, S.; Piot, M.; Issoufaly, I.; Giraud, ...",2023,The aim of the data farming project by the Uni...,1
481,10.3389/frai.2023.1131667,EPPO ontology: a semantic-driven approach for ...,"Ayllon-Benitez, Aaron; Bernabe-Diaz, Jose Anto...",2023,The agricultural industry and regulatory organ...,1
482,10.12688/f1000research.126463.2,A2TEA: Identifying trait-specific evolutionary...,"Stocker, Tyll; Uebermuth-Feldhaus, Carolin; Bo...",2022,Background: Plants differ in their ability to ...,1
483,10.1080/02626667.2021.1890749,A methodology to downscale water demand data w...,"Zogheib, Charles; Ochoa-Tocachi, Boris F.; Mou...",2021,Mountainous regions are a hotspot for water sc...,1


# Final Dataset

In [11]:
df_all = pd.concat([df_ieee, df_scopus, df_wos], ignore_index=True)
df_all

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/JSTARS.2015.2417832,The Impact of National Land Cover and Soils Da...,A. Pacheco; H. McNairn; A. Mahmoodi; C. Champa...,2015,"To ensure sustainable agriculture production, ...",14.0
1,10.1109/IGARSS.2018.8518014,Evaluation of Land Surface Model Against Smap ...,K. Das; J. Singh; J. Hazra; S. Kalyanaraman,2018,Soil moisture and temperature are key inputs t...,5.0
2,10.1109/TEMSMET51618.2020.9557403,A Comprehensive Survey of Classification Algor...,C. Chandana; G. Parthasarathy,2020,As we are aware that agriculture is one of the...,8.0
3,10.1109/IDAACS.2017.8095109,Land degradation estimation from global and na...,N. Kussul; A. Kolotii; A. Shelestov; B. Yailym...,2017,"In this paper, we investigate global and natio...",11.0
4,10.1109/MetroAgriFor52389.2021.9628717,Improving accuracy and efficiency in plant det...,L. Ohnemüller; A. Briassouli,2021,Detecting plants in images is central in preci...,2.0
...,...,...,...,...,...,...
894,10.1016/j.canrad.2023.06.016,Real world data in radiotherapy: A data farmin...,"Guihard, S.; Piot, M.; Issoufaly, I.; Giraud, ...",2023,The aim of the data farming project by the Uni...,1.0
895,10.3389/frai.2023.1131667,EPPO ontology: a semantic-driven approach for ...,"Ayllon-Benitez, Aaron; Bernabe-Diaz, Jose Anto...",2023,The agricultural industry and regulatory organ...,1.0
896,10.12688/f1000research.126463.2,A2TEA: Identifying trait-specific evolutionary...,"Stocker, Tyll; Uebermuth-Feldhaus, Carolin; Bo...",2022,Background: Plants differ in their ability to ...,1.0
897,10.1080/02626667.2021.1890749,A methodology to downscale water demand data w...,"Zogheib, Charles; Ochoa-Tocachi, Boris F.; Mou...",2021,Mountainous regions are a hotspot for water sc...,1.0


In [12]:
df_all = df_all.drop_duplicates(subset=['DOI'])
df_all = df_all.drop(columns=['Citations'])
df_all = df_all.reset_index(drop=True)
df_all

,DOI,Title,Authors,Year,Abstract
0,10.1109/JSTARS.2015.2417832,The Impact of National Land Cover and Soils Da...,A. Pacheco; H. McNairn; A. Mahmoodi; C. Champa...,2015,"To ensure sustainable agriculture production, ..."
1,10.1109/IGARSS.2018.8518014,Evaluation of Land Surface Model Against Smap ...,K. Das; J. Singh; J. Hazra; S. Kalyanaraman,2018,Soil moisture and temperature are key inputs t...
2,10.1109/TEMSMET51618.2020.9557403,A Comprehensive Survey of Classification Algor...,C. Chandana; G. Parthasarathy,2020,As we are aware that agriculture is one of the...
3,10.1109/IDAACS.2017.8095109,Land degradation estimation from global and na...,N. Kussul; A. Kolotii; A. Shelestov; B. Yailym...,2017,"In this paper, we investigate global and natio..."
4,10.1109/MetroAgriFor52389.2021.9628717,Improving accuracy and efficiency in plant det...,L. Ohnemüller; A. Briassouli,2021,Detecting plants in images is central in preci...
...,...,...,...,...,...
627,10.1016/j.dib.2024.110225,An agro-physiological dataset on industrial to...,"Carucci, Federica; Bregaglio, Simone; Caldarol...",2024,The availability of field experimental data pl...
628,10.3389/fsufs.2023.1216205,Global hotspots of climate change adaptation a...,"Costa Jr, Ciniro; Thornton, Philip; Wollenberg...",2023,As countries prioritize climate action under t...
629,10.1016/j.canrad.2023.06.016,Real world data in radiotherapy: A data farmin...,"Guihard, S.; Piot, M.; Issoufaly, I.; Giraud, ...",2023,The aim of the data farming project by the Uni...
630,10.1080/02626667.2021.1890749,A methodology to downscale water demand data w...,"Zogheib, Charles; Ochoa-Tocachi, Boris F.; Mou...",2021,Mountainous regions are a hotspot for water sc...


# Abstract Evaluation with Gemini

In [13]:
load_dotenv()

MODEL= os.getenv("MODEL")
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=API_KEY)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [14]:
class BatchReviewItem(BaseModel):
    id: int
    decision: Annotated[Literal["yes", "no", "maybe"], Field(
        description="Decision about including the paper:"
                    "- 'yes': There is clear evidence that the paper uses, shares, or discusses datasets, and it fully meets ALL inclusion criteria while NOT violating ANY exclusion criteria."
                    "- 'no': There is clear evidence that the paper should be excluded because it fails to meet all inclusion criteria or it meets at least one exclusion criterion."
                    "- 'maybe': The information is insufficient or ambiguous; checking the full text is recommended."
                    "If unsure, or it is needed to check other infos (like the full text) prefer 'maybe' over 'no'.")]
    reason: Annotated[str, Field(description="Short explanation for the decision")]
    data_source: Annotated[str, Field(
        description="Official and recognized name of the data source used in the paper if the dataset is explicitly mentioned."
                    "The dataset must be a real, identifiable resource (dataset, database, or data collection); check online if necessary."
                    "If only a vague or generic description is provided (e.g., 'agricultural data' or 'crop dataset' or 'generated data'), leave it empty."
                    "Use only the proper name of the resource, without adding descriptors about the type of data source (e.g., use 'PlantVillage' instead of 'PlantVillage dataset')."
                    "If multiple data sources are mentioned, insert all of them in the same string separated by ';' (semicolon)."
                    "If the decision is 'no', always leave it empty.")]


class BatchReview(BaseModel):
    items: list[BatchReviewItem]

fallback = BatchReviewItem(id=-1, decision="maybe", reason="No response", data_source="")


def is_open_data_relevant_batch(df_chunk: pd.DataFrame, client, model=None, max_retries=3):
    if model is None:
        model = MODEL if MODEL else "gemini-2.5-flash-lite"

    # Prepare record strings for prompt
    records = []
    for idx, row in df_chunk.iterrows():
        title = "" if pd.isna(row.get("Title")) else str(row.get("Title"))
        abstract = "" if pd.isna(row.get("Abstract")) else str(row.get("Abstract"))
        records.append(f"- id: {int(idx)}\n  Title: {title}\n  Abstract: {abstract}")

    # Build prompt with inclusion/exclusion criteria and instructions
    prompt = (
            "Evaluate each record based on the following Inclusion and Exclusion Criteria and return ONLY JSON that matches the provided schema.\n\n"
            "Inclusion Criteria:\n"
            "(a) The abstract mentions the use or production of datasets, databases, or other data sources.\n"
            "(b) The data sources are related to agriculture or closely related fields (e.g.: climate, soil...).\n\n"
            "Exclusion Criteria:\n"
            "(a) The abstract states that the data are not available or must be requested from the authors.\n"
            "(b) The data used or generated are not free (cost is required).\n"
            "(c) The data are local/regional/national only (not global coverage).\n\n"
            "ALL the requirements should be met.\n\n"
            "Output Instructions:\n"
            "- For each provided record, return an object with: id, decision ('yes'|'no'|'maybe'), reason, data_source.\n"
            "- If title or abstract are missing or empty use decision='no' and reason='Title or abstract is missing', data_source=''.\n\n"
            "Records to evaluate:\n"
            + "\n".join(records)
    )

    global _sent_calls_count
    global _completed_evaluations_count
    backoff = 10

    # Error management with exponential backoff
    for _ in range(max_retries):
        _rate_limit_block_until_allowed()
        try:
            _sent_calls_count += 1
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": BatchReview,
                    "temperature": 0.0
                },
            )
            # Update total evaluations count
            try:
                _completed_evaluations_count += len(df_chunk)
            except Exception as e:
                print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
                _completed_evaluations_count = len(df_chunk)
            print(f"[GEMINI] Completed {_completed_evaluations_count} total evaluations.")
            return resp.parsed.items
        except Exception as e:
            print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
            time.sleep(backoff)
            backoff *= 2
    return []


def _rate_limit_block_until_allowed():
    # Block until the number of calls in the window is below max
    while True:
        now = time.monotonic()
        while _rate_calls_log and (now - _rate_calls_log[0]) >= _RATE_WINDOW_SEC:
            _rate_calls_log.popleft()
        if len(_rate_calls_log) < _RATE_MAX_CALLS:
            _rate_calls_log.append(now)
            return


# Rate limiter: max 2 requests every 60 seconds
_RATE_WINDOW_SEC = 60.0
_RATE_MAX_CALLS = 2
_rate_calls_log = deque()

batch_size = 20  # number of papers per batch
df_evaluated = df_all.copy()

_sent_calls_count = 0
_completed_evaluations_count = 0

# Process dataframe in batches
for start in range(0, len(df_evaluated), batch_size):
    chunk = df_evaluated.iloc[start:start + batch_size]
    items = is_open_data_relevant_batch(chunk, client, model=MODEL)
    mapping = {int(it.id): it for it in items} if items else {}

    decisions = []
    reasons = []
    sources = []
    for idx in chunk.index:
        it = mapping.get(int(idx))
        if it is None:
            decisions.append(fallback.decision)
            reasons.append(fallback.reason)
            sources.append(fallback.data_source)
        else:
            decisions.append(it.decision)
            reasons.append(it.reason)
            sources.append(it.data_source)

    # Update dataframe with results
    df_evaluated.loc[chunk.index, 'Include'] = decisions
    df_evaluated.loc[chunk.index, 'Reason'] = reasons
    df_evaluated.loc[chunk.index, 'Data Source'] = sources

df_evaluated[['DOI', 'Include', 'Reason', 'Data Source']]

[GEMINI] Completed 20 total evaluations.
[GEMINI] Completed 40 total evaluations.
[GEMINI] Completed 60 total evaluations.
[GEMINI] Completed 80 total evaluations.
[GEMINI] Completed 100 total evaluations.
[GEMINI] Completed 120 total evaluations.
[GEMINI] Completed 140 total evaluations.
[GEMINI] Completed 160 total evaluations.
[GEMINI] Completed 180 total evaluations.
[GEMINI] Completed 200 total evaluations.
[GEMINI] Completed 220 total evaluations.
[GEMINI] Completed 240 total evaluations.
[GEMINI] Completed 260 total evaluations.
[GEMINI] Completed 280 total evaluations.
[GEMINI] Completed 300 total evaluations.
[GEMINI] Completed 320 total evaluations.
[GEMINI] Completed 340 total evaluations.
[GEMINI] Completed 360 total evaluations.
[GEMINI] Completed 380 total evaluations.
[GEMINI] Completed 400 total evaluations.
[GEMINI] Completed 420 total evaluations.
[GEMINI] Completed 440 total evaluations.
[GEMINI] Completed 460 total evaluations.
[GEMINI] Completed 480 total evaluatio

,DOI,Include,Reason,Data Source
0,10.1109/JSTARS.2015.2417832,no,The study explicitly uses national datasets (C...,
1,10.1109/IGARSS.2018.8518014,yes,The abstract mentions the use of several globa...,SMAP;NL-DAS;GLDAS;Global Data Assimilation Sys...
2,10.1109/TEMSMET51618.2020.9557403,no,The abstract is for a survey paper and refers ...,
3,10.1109/IDAACS.2017.8095109,yes,The abstract mentions the use of global datase...,ESA CCI-LC;Land Productivity Dynamics;Landsat
4,10.1109/MetroAgriFor52389.2021.9628717,maybe,The abstract mentions the 'Open Plant Phenotyp...,Open Plant Phenotyping Database;MS COCO
...,...,...,...,...
627,10.1016/j.dib.2024.110225,no,The dataset presented is explicitly regional (...,
628,10.3389/fsufs.2023.1216205,yes,The abstract states the study evaluates global...,
629,10.1016/j.canrad.2023.06.016,no,The data described is related to radiotherapy ...,
630,10.1080/02626667.2021.1890749,no,The dataset produced is for a specific region ...,


In [15]:
df_evaluated.to_csv("data/global/df_evaluated_global.csv", index=False)